# Homework 08: Regression Inference, Diagnostics, and Classification

This assignment is due Thursday, November 16 at 11:59PM. You will receive an early submission bonus point if you turn it in by Wednesday, November 15 at 11:59PM. Directly sharing answers is not okay, but discussing problems with course staff or with other students is encouraged. Passing the automatic tests does not guarantee full credit on any question. The tests are provided to help catch some common errors, but it is your responsibility to answer the questions correctly.


The final problem of this homework, the classification portion, is optional and will not count for or against your grade.

Reading:
- Textbook chapter [13](https://www.inferentialthinking.com/chapters/13/prediction.html)
- Textbook chapter [14](https://www.inferentialthinking.com/chapters/14/inference-for-regression.html)

Run the cell below to prepare the notebook.

In [1]:
# Run this cell to set up the notebook, but please don't change it.
import numpy as np
from datascience import *

# These lines do some fancy plotting magic.
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import warnings
warnings.simplefilter('ignore', FutureWarning)
from matplotlib import patches
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

# Don't change this cell; just run it. 
from client.api.notebook import Notebook
ok = Notebook('hw08.ok')
ok.auth(inline=True);

## Quantifying Sampling Errors in Regression

Previously, in this class we've used confidence intervals to quantify uncertainty about estimates as well as to test predictions. To run a hypothesis test using a confidence interval, we use the following procedure:
1. Formulate a null hypothesis
2. Formulate an alternative hypothesis 
3. Choose a test statistic and compute the observed value for the test statistic
4. Bootstrap, finding a value of the test stat for each resample
5. Generate a 95% confidence interval from those resampled test stats
6. Based on whether your value is in an interval, make a conclusion

Another thing we've covered recently is the use of linear regression to make predictions, using correlated variables. An example is, say, predicting the height of children based on the heights of their parents.

Some important formulas for regression follow:

- $r = \text{mean}(x_{su} * y_{su})$
- $\text{slope} = r * \frac{\text{SD of }y}{\text{SD of }x}$
- $\text{intercept} = \text{average of }y − \text{slope}\cdot\text{average of }x$

We can combine these two topics together in order to make even more powerful statements about our population given just a sample as before. We can use the following techniques to do so:
- Bootstrapped interval for the true slope
- Bootstrapped prediction interval for y (given a particular value of x)

This homework further explores these two advanced methods.

Recall the Old Faithful dataset from our lab on regression. The table contains two pieces of information about each eruption of the Old Faithful geyser in Yellowstone National Park:
1. The duration of the eruption, in minutes.
2. The time between this eruption and the next eruption (the "waiting time"), in minutes.

The dataset is plotted below along with its line of best fit.

In [2]:
faithful = Table.read_table('faithful_inference.csv')
faithful.scatter('duration', fit_line=True)
faithful

Two quick questions:
- What variable are we trying to predict? What variable are we given?
<sub><sup>We are trying to predict waiting time. We are given duration.</sup></sub>

- Given the regression line above and that the wait time before that was 60 minutes, can we say that the eruption likely to have lasted 2.5 minutes? <sub><sup>No. This regression line predicts y using x. To go from x to y we would need to create a different regression line.</sup></sub>


### Finding the Bootstrap Confidence Interval for the True Slope

Last time we looked at this dataset, we noticed the apparent linear relationship betwen duration and wait, and we decided to use regression to predict wait in terms of duration. However, our data are just a sample of all the eruptions that have happened at Old Faithful. As we know, relationships can appear in a sample that don't really exist in the population from which the sample was taken.

Before we move forward using our linear model, we would like to know whether or not there truly exists a relationship between duration and wait time. If there is no relationship between the two, then we'd expect a correlation of 0, which would give us a slope of 0. Now, write in null and alternative hypotheses, based on your knowledge of hypothesis tests you've conducted in the past.

- **Null Hypothesis:** [*Your solution goes here*]
- **Alternate Hypothesis:** [*Your solution goes here*]

We will use the method of confidence intervals to test this hypothesis.

<div class="hide">\pagebreak</div>
#### Question 1
We'll warm up by implementing some familiar functions. You may use these functions throughout this assignment. Start by defining these two functions:

1. `standard_units` should take in an array of numbers and return an array containing those numbers converted to standard units.
2. `correlation` should take in a table with 2 columns and return the correlation between these columns. Hint: you may want to use the `standard_units` function you defined above.

In [3]:
def standard_units(arr):
    ...

def correlation(tbl):
    ...

In [7]:
_ = ok.grade('q1_1')

<div class="hide">\pagebreak</div>
#### Question 2
Using the functions you just implemented, create a function called `fit_line`.  It should take a table as its argument.  It should return an array containing the slope and intercept of the regression line that predicts the second column in the table using the first.

In [8]:
def fit_line(tbl):
    ...
    slope = ...
    intercept = ...
    return make_array(slope, intercept)

# This should compute the slope and intercept of the regression
# line predicting wait time from duration in the faithful dataset.
fit_line(faithful)

In [15]:
# Ensure your fit_line function fits a reasonable line 
# to the data in faithful, using the plot below

slope, intercept = fit_line(faithful)
faithful.scatter(0)
plt.plot([min(faithful[0]), max(faithful[0])], 
         [slope*min(faithful[0])+intercept, slope*max(faithful[0])+intercept])
plt.show()

In [19]:
_ = ok.grade('q1_2')

Now we have all the tools we need in order to create a confidence interval quantifying our uncertainty about the true relationship between duration and wait time.

<div class="hide">\pagebreak</div>
#### Question 3
Use the bootstrap to compute 1000 resamples from our dataset. For each resample, compute the slope of the best fit line. Put these slopes in the array `resample_slopes`, giving you the empirical distribution of regression line slopes in resamples.

In [16]:
resample_slopes = make_array()
for i in np.arange(1000):
    sample = ...
    resample_line = ...
    resample_slope = ...
    resample_slopes = ...

Table().with_column("Slope estimate", resample_slopes).hist() # DO NOT CHANGE THIS LINE

<div class="hide">\pagebreak</div>
#### Question 4
Use your resampled slopes to construct an approximate 95% confidence interval for the true value of the slope.

In [18]:
lower_end = ...
upper_end = ...
print("95% confidence interval for slope: [{:g}, {:g}]".format(lower_end, upper_end))

<div class="hide">\pagebreak</div>
#### Question 5
Based on your confidence interval, would you accept or reject the null hypothesis that the true slope is 0?  Why?  What P-value cutoff are you using?

### Finding the Bootstrap Prediction Interval

Suppose we're tourists at Yellowstone, and we'd like to know how long we'll have to wait for the next Old Faithful eruption.  We decide to use our regression line to make some predictions for the waiting times.  But just as we're uncertain about the slope of the true regression line, we're also uncertain about the predictions we'd make based on the true regression line.

<div class="hide">\pagebreak</div>
#### Question 6
Define the function `fitted_value`.  It should take 2 arguments:

1. A table with 2 columns.  We'll be predicting the values in the second column using the first.
2. A number, the value of the predictor variable for which we'd like to make a prediction.

Make sure to use your `fit_line` function. 

In [23]:
def fitted_value(table, given_x):
    # The staff solution took 4 lines of code.
    ...

# Here's an example of how fitted_value is used.  This should
# compute the prediction for the wait time of an eruption that lasts 
# two minutes .
two_minutes_wait = fitted_value(faithful, 2)
two_minutes_wait

In [26]:
_ = ok.grade('q1_6')

<div class="hide">\pagebreak</div>
#### Question 7
The park ranger tells us that the most recent eruption lasted 5 minutes. Using your function above, assign the variable `five_minutes_wait` to the predicted wait time. 

In [27]:
five_minutes_wait = ...
five_minutes_wait

In [28]:
_ = ok.grade('q1_7')

Juan, a fellow tourist, raises the following objection to your prediction:

> "Your prediction depends on your sample of 272 eruptions.  Couldn't your prediction have been different if you had happened to have a different sample of eruptions?"

Having read [section 14.3](https://www.inferentialthinking.com/chapters/14/3/prediction-intervals.html) of the textbook, you know just the response!

<div class="hide">\pagebreak</div>
#### Question 8
Define the function `bootstrap_lines`.  It should take two arguments:
1. A table with two columns.  As usual, we'll be predicting the second column using the first.
2. An integer, a number of bootstraps to run.

It should return a *table* whose first column, `"Slope"`, contains the given number of bootstrapped slopes, and whose second column, `"Intercept"`, contains the corresponding bootstrapped intercepts.  Each slope and intercept should come from a regression line that predicts column 2 from column 1 of a resample of the given table.  The table should have 1 row for each bootstrap replication.

**Hint:** Your code should look very similar to the code you wrote for question 3, with just a few key changes.

In [39]:
def bootstrap_lines(tbl, num_bootstraps):
    ...

# When you're done, this code should produce the slopes
# and intercepts of 1000 regression lines computed from
# resamples of the faithful table.
regression_lines = bootstrap_lines(faithful, 1000)
regression_lines

<div class="hide">\pagebreak</div>
#### Question 9
Create an array called `predictions_for_five`.  It should contain 1000 numbers.  Each number should be the predicted waiting time after an eruption with a duration of 5 minutes, using a different bootstrapped regression line. Hint: use `regression_lines` from the previous questions.

In [41]:
predictions_for_five = ...

# This will make a histogram of your predictions:
table_of_predictions = Table().with_column('Predictions at eruptions=5', predictions_for_five)
table_of_predictions.hist('Predictions at eruptions=5', bins=20)

<div class="hide">\pagebreak</div>
#### Question 10
Create a 95 percent confidence interval for these predictions.

In [47]:
lower_bound = ...
upper_bound = ...

print('95% Confidence interval for predictions for x=5: (', lower_bound,",", upper_bound, ')')

<div class="hide">\pagebreak</div>
#### Question 11
Look at the scatter plot of the data at the start of this exercise.  Does your confidence interval cover around 95 percent of eruptions in `faithful` that had an eruption duration of 5 minutes? If not, what does this confidence interval mean?

## 2. Visual Diagnostics for Linear Regression


### Regression Model Diagnostics
Linear regression isn't always the best way to describe the relationship between two variables. We'd like to develop techniques that will help us decide whether or not to use a linear model to predict one variable based on another.

We will use the insight that if a regression fits a set of points well, then the residuals from that regression line will show no pattern when plotted against the predictor variable. 

The table below contains information about crime rates and median home values in suburbs of Boston. We will attempt to use linear regression to predict median home value in terms of crime rate.

#### About the dataset
All data are from 1970.  Crime rates are per capita per year; home values are in thousands of dollars.  The crime data come from the FBI, and home values are from the US Census Bureau.  

Run the next cell to load the data and see a scatter plot.

In [2]:
boston = Table.read_table('boston_housing.csv')
boston.scatter('Crime Rate')

<div class="hide">\pagebreak</div>
#### Question 1 Finding Residuals
Write a function called `residuals`.  It should take a single argument, a table.  It should first compute the slope and intercept of the regression line that predicts the second column of that table (accessible as `tbl.column(1)`) using the first column (`tbl.column(0)`).  `residuals` should return an array containing the *residuals* for that regression line. Recall that residuals are given by 

$$\texttt{residual} = \texttt{observed value} - \texttt{regression estimate}.$$

Hint: You may want to use your function fitline from earlier

In [3]:
def residuals(tbl):
    ...

In [5]:
_ = ok.grade('q2_1')
_ = ok.backup()

<div class="hide">\pagebreak</div>
#### Question 2  Plotting a Residual Plot
Make a scatter plot of the residuals for the Boston housing dataset against crime rate. Crime rate should be on the horizontal axis.

In [ ]:
...

<div class="hide">\pagebreak</div>
#### Question 3 Interpreting the Residual Plot
Does the plot of residuals look roughly like a formless cloud? Or is there some kind of pattern in them? Are they centered around 0?

*Write your answer here, replacing this text.*

<div class="hide">\pagebreak</div>
#### Question 4 Is Linear Regression a good fit?
Does it seem like a linear model is appropriate for describing the relationship between crime and median home value? Explain your reasoning.

*Write your answer here, replacing this text.*

[Section 13.6](https://www.inferentialthinking.com/chapters/13/6/numerical-diagnostics.html) of the textbook describes some mathematical facts that hold for all regression estimates, regardless of goodness of fit.  One fact is that there is a relationship between the standard deviation of the residuals, the standard deviation of the response variable, and the correlation.  Let us test this.

Below, we have imported a new table, the Old Faithful data.

In [15]:
old_faithful = Table.read_table('faithful.csv')
old_faithful

The following cell makes a residual plot for this new dataset.

In [22]:
Table().with_columns('Residual', residuals(old_faithful), 'Duration', old_faithful.column('duration')).scatter('Duration')

<div class="hide">\pagebreak</div>
#### Question 5 Finding the Standard Deviation of Residuals for Boston dataset
Directly compute the standard deviation of the residuals from the Boston data.  Then compute the same quantity without using the residuals, using the formula described in section 13.6 instead.

In [23]:
boston_residual_sd = ...
boston_residual_sd_from_formula = ...

print("Residual SD: {0}".format(boston_residual_sd))
print("Residual SD from the formula: {0}".format(boston_residual_sd_from_formula))

In [24]:
_ = ok.grade('q2_5')
_ = ok.backup()

<div class="hide">\pagebreak</div>
#### Question 6 Finding the Standard Deviation of Residuals for 'Old_Faithful' dataset
Repeat the procedure from Question 5 for the `old_faithful` dataset.

In [25]:
faithful_residual_sd = ...
faithful_residual_sd_from_formula = ...

print("Residual SD: {0}".format(faithful_residual_sd))
print("Residual SD from the formula: {0}".format(faithful_residual_sd_from_formula))

In [26]:
_ = ok.grade('q2_6')
_ = ok.backup()

## Optional: Reading Sign Language with Classification

Brazilian Sign Language is a visual language used primarily by Brazilians who are deaf.  It is more commonly called Libras.  People who communicate with visual language are called *signers*.  Here is a video of someone signing in Libras:

In [9]:
from IPython.lib.display import YouTubeVideo
YouTubeVideo("mhIcuMZmyWM")

Programs like Siri or Google Now begin the process of understanding human speech by classifying short clips of raw sound into basic categories called *phones*.  For example, the recorded sound of someone saying the word "robot" might be broken down into several phones: "rrr", "oh", "buh", "aah", and "tuh".  Phones are then grouped together into further categories like words ("robot") and sentences ("I, for one, welcome our new robot overlords") that carry more meaning.

A visual language like Libras has an analogous structure.  Instead of phones, each word is made up of several *hand movements*.  As a first step in interpreting Libras, we can break down a video clip into small segments, each containing a single hand movement.  The task is then to figure out what hand movement each segment represents.

We can do that with classification!

The [data](https://archive.ics.uci.edu/ml/machine-learning-databases/libras/movement_libras.names) in this exercise come from Dias, Peres, and Biscaro, researchers at the University of Sao Paulo in Brazil.  They identified 15 distinct hand movements in Libras (probably an oversimplification, but a useful one) and captured short videos of signers making those hand movements.  (You can read more about their work [here](http://ieeexplore.ieee.org/Xplore/login.jsp?url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F5161636%2F5178557%2F05178917.pdf&authDecision=-203). The paper is gated, so you will need to use your institution's Wi-Fi or VPN to access it.)

For each video, they chose 45 still frames from the video and identified the location (in horizontal and vertical coordinates) of the signer's hand in each frame.  Since there are two coordinates for each frame, this gives us a total of 90 numbers summarizing how a hand moved in each video.  Those 90 numbers will be our *attributes*.

Each video is *labeled* with the kind of hand movement the signer was making in it.  Each label is one of 15 strings like "horizontal swing" or "vertical zigzag".

For simplicity, we're going to focus on distinguishing between just two kinds of movements: "horizontal straight-line" and "vertical straight-line".  We took the Sao Paulo researchers' original dataset, which was quite small, and used some simple techniques to create a much larger synthetic dataset.

These data are in the file `movements.csv`.  Run the next cell to load it.

In [10]:
movements = Table.read_table("movements.csv")
movements.take(np.arange(5))

The cell below displays movements graphically.  Run it and use the slider to answer the next question.

In [11]:
# Just run this cell and use the slider it produces.
def display_whole_movement(row_idx):
    num_frames = int((movements.num_columns-1)/2)
    row = np.array(movements.drop("Movement type").row(row_idx))
    xs = row[np.arange(0, 2*num_frames, 2)]
    ys = row[np.arange(1, 2*num_frames, 2)]
    plt.figure(figsize=(5,5))
    plt.plot(xs, ys, c="gold")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.xlim(-.5, 1.5)
    plt.ylim(-.5, 1.5)
    plt.gca().set_aspect('equal', adjustable='box')

def display_hand(example, frame, display_truth):
        time_idx = frame-1
        display_whole_movement(example)
        x = movements.column(2*time_idx).item(example)
        y = movements.column(2*time_idx+1).item(example)
        plt.annotate(
            "frame {:d}".format(frame),
            xy=(x, y), xytext=(-20, 20),
            textcoords = 'offset points', ha = 'right', va = 'bottom',
            color='white',
            bbox = {'boxstyle': 'round,pad=0.5', 'fc': 'black', 'alpha':.4},
            arrowprops = {'arrowstyle': '->', 'connectionstyle':'arc3,rad=0', 'color': 'black'})
        plt.scatter(x, y, c="black", zorder=10)
        plt.title("Hand positions for movement {:d}{}".format(example, "\n(True class: {})".format(movements.column("Movement type").item(example)) if display_truth else ""))

def animate_movement():
    interact(
        display_hand,
        example=widgets.BoundedIntText(min=0, max=movements.num_rows-1, value=0, msg_throttle=1),
        frame=widgets.IntSlider(min=1, max=int((movements.num_columns-1)/2), step=1, value=1, msg_throttle=1),
        display_truth=fixed(False))

animate_movement()

<div class="hide">\pagebreak</div>
#### Question 1
Before we move on, check your understanding of the dataset.  Judging by the plot, is the first movement example a vertical motion, or a horizontal motion, or is it difficult to tell?  This is the kind of question a classifier has to answer.  Find out the right answer by looking at the `"Movement type"` column.  (It's okay if you guessed wrong for this one.)

*Write your answer here, replacing this text.*

### Splitting the dataset
We'll do 2 different kinds of things with the `movements` dataset:
1. We'll build a classifier that uses the movements with known labels as examples to classify similar movements.  This is called *training*.
2. We'll evaluate or *test* the accuracy of the classifier we build.

For reasons discussed in lecture and the textbook, we want to use separate datasets for these two purposes.  So we split up our one dataset into two.

<div class="hide">\pagebreak</div>
#### Question 2
Create a table called `train_movements` and another table called `test_movements`.  `train_movements` should include the first $\frac{11}{16}$ of the rows in `movements` (rounded to the nearest integer), and `test_movements` should include the remaining $\frac{5}{16}$.

*Hint:* Use the table method `take`.

In [12]:
training_proportion = 11/16
num_movements = movements.num_rows
num_train = int(round(num_movements * training_proportion))

train_movements = ...
test_movements = ...

print("Training set:\t",   train_movements.num_rows, "examples")
print("Test set:\t",       test_movements.num_rows, "examples")

In [14]:
_ = ok.grade('q3_2')
_ = ok.backup()

### Using only 2 features
First let's see how well we can distinguish two movements (a vertical line and a horizontal line) using the hand position from just a single frame (without the other 44).

<div class="hide">\pagebreak</div>
#### Question 3
Make a table called `train_two_features` with only 3 columns: the first frame’s x coordinate and first frame’s y coordinate are our chosen features, as well as the movement type; only the examples in `train_movements`. 

In [15]:
train_two_features = ...
train_two_features

In [16]:
_ = ok.grade('q3_3')
_ = ok.backup()

Now we want to make a scatter plot of the frame coordinates, where the dots for horizontal straight-line movements have one color and the dots for vertical straight-line movements have another color.  Here is a scatter plot without colors:

In [17]:
train_two_features.scatter("Frame 1 x", "Frame 1 y")

This isn't useful because we don't know which dots are which movement type.  We need to tell Python how to color the dots.  Let's use gold for vertical and blue for horizontal movements.

`scatter` takes an extra argument called `colors` that's the name of an extra column in the table that contains colors (strings like "red" or "orange") for each row.  So we need to create a table like this:

|Frame 1 x|Frame 1 y|Movement type|Color|
|-|-|-|-|
|0.522768|0.769731|vertical straight-line|gold|
|0.179546|0.658986|horizontal straight-line|blue|
|...|...|...|...|

<div class="hide">\pagebreak</div>
#### Question 4
In the cell below, create a table named `with_colors`.  It should have the same columns as the example table above, but with a row for each row in `train_two_features`.

In [18]:
# You should find the following table useful.
type_to_color = Table().with_columns(
    "Movement type", make_array("vertical straight-line", "horizontal straight-line"),
    "Color",         make_array("gold",                   "blue"))

with_colors = ...
with_colors.scatter("Frame 1 x", "Frame 1 y", colors="Color")

<div class="hide">\pagebreak</div>
#### Question 5
Based on the scatter plot, how well will a nearest-neighbor classifier based on only these 2 features (the x- and y-coordinates of the hand position in the first frame) work?  Will it:

1. distinguish almost perfectly between vertical and horizontal movements;
2. distinguish somewhat well between vertical and horizontal movements, getting some correct but missing a substantial proportion; or
3. be basically useless in distinguishing between vertical and horizontal movements?

Why?

*Write your answer here, replacing this text.*

## 4. Submission


Once you're finished, select "Save and Checkpoint" in the File menu and then execute the `submit` cell below. The result will contain a link that you can use to check that your assignment has been submitted successfully. If you submit more than once before the deadline, we will only grade your final submission.

In [ ]:
_ = ok.submit()